Data Collection for I320D ML Project. Looking for financial data that will enable us to an individual stock crash. Will need time series data for price, company actions, finances, and macroeconomic indicators as factors. 

Data Collection Periods: 01/01/2005 - 01/01/2010 and 01/01/2020 - 03/30/2023. The first period will primarily be used as a training set and the more recent period will be used for testing.

### NBER Recession Indicators for the United States

These are daily indicators of whether or not the United States is currently in a recession, with 0 = no and 1 = yes. Downloaded from the St. Louis Fed FRED Economic Database. These values will likely be used as our main labels.

In [13]:
import pandas as pd

us_rec = pd.read_csv(r'Macroeconomic_Data\05_10_USRECD.csv')
us_rec

,DATE,USRECD
0,2005-01-01,0
1,2005-01-02,0
2,2005-01-03,0
3,2005-01-04,0
4,2005-01-05,0
...,...,...
1822,2009-12-28,0
1823,2009-12-29,0
1824,2009-12-30,0
1825,2009-12-31,0


## Collection of Further Macroeconomic Indicators

- T10YIE: 10-Year Breakeven Inflation Rate
- T10Y2Y: 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity
- DFF: Federal Funds Effective Rate
- SP500: S&P 500
- VIXCLS: Volatility Index

### Collecting Security Data

Daily data for the SVB stock:

In [18]:
import yfinance as yf

svb_p1 = yf.download("SIVB", start="2005-01-01", end="2010-01-01")
svb_p1.to_csv("svb_p1.csv")
svb_p1

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2005-01-03,44.580002,45.259998,44.459999,44.799999,44.799999,453265
2005-01-04,44.799999,45.060001,44.029999,44.200001,44.200001,368711
2005-01-05,44.040001,44.490002,43.820000,43.910000,43.910000,663902
2005-01-06,44.099998,44.500000,43.889999,44.200001,44.200001,349635
2005-01-07,44.330002,44.529999,43.380001,43.430000,43.430000,300137
...,...,...,...,...,...,...
2009-12-24,41.810001,42.470001,41.590000,42.430000,42.430000,156518
2009-12-28,42.340000,42.680000,41.720001,41.950001,41.950001,303056
2009-12-29,42.080002,42.480000,41.849998,42.220001,42.220001,245109


In [20]:
svb_p2 = yf.download("SIVB", start="2020-01-01", end="2023-03-31")
svb_p2.to_csv("svb_p2.csv")
svb_p2

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,252.649994,254.279999,249.669998,254.270004,254.270004,242917
2020-01-03,247.960007,251.119995,246.800003,250.330002,250.330002,476454
2020-01-06,246.839996,249.800003,245.059998,249.240005,249.240005,491021
2020-01-07,248.660004,251.119995,247.580002,250.399994,250.399994,501982
2020-01-08,249.820007,254.115005,249.130005,252.779999,252.779999,502032
...,...,...,...,...,...,...
2023-03-24,106.040001,106.040001,106.040001,106.040001,106.040001,0
2023-03-27,106.040001,106.040001,106.040001,106.040001,106.040001,0
2023-03-28,0.530000,0.740000,0.010000,0.400000,0.400000,84502118


## Feature Engineering

First, adding a column for the percentile of the price in the last 30 days, judging price by the adjusted close price.

In [48]:
from scipy import stats
import numpy as np

# Change this variable assignment to rerun on a different df
df = svb_p1

close = df['Adj Close']
percentiles = []

for i, val in enumerate(close):

    # Selecting last 30 days of prices
    last_30 = close[i-30: i]

    # Calculating percentile of current price in respect to the last 30
    if np.any(last_30):
        median = stats.percentileofscore(last_30, close[i])
        percentiles.append(median)
    else:
        percentiles.append(np.NaN)

df["percentile_last_30"] = percentiles
df

,Open,High,Low,Close,Adj Close,Volume,percentile_last_30
Date,,,,,,,
2005-01-03,44.580002,45.259998,44.459999,44.799999,44.799999,453265,NaN
2005-01-04,44.799999,45.060001,44.029999,44.200001,44.200001,368711,NaN
2005-01-05,44.040001,44.490002,43.820000,43.910000,43.910000,663902,NaN
2005-01-06,44.099998,44.500000,43.889999,44.200001,44.200001,349635,NaN
2005-01-07,44.330002,44.529999,43.380001,43.430000,43.430000,300137,NaN
...,...,...,...,...,...,...,...
2009-12-24,41.810001,42.470001,41.590000,42.430000,42.430000,156518,96.666667
2009-12-28,42.340000,42.680000,41.720001,41.950001,41.950001,303056,90.000000
2009-12-29,42.080002,42.480000,41.849998,42.220001,42.220001,245109,93.333333
